In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
import QUANTAXIS as QA
import talib as ta
import datetime #, time

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [38]:
data=QA.QAFetch.QATdx.QA_fetch_get_index_min('000001','1990-01-01','2018-11-23','60min')

In [3]:
data = pd.read_excel('index_60min.xlsx',index_col=0, header=0)

In [40]:
# data = data.loc[:,['open', 'close', 'high', 'low', 'vol', 'amount', 'up_count',
#        'down_count', 'code', 'date', 'date_stamp', 'time_stamp', 'type']]

In [44]:
# data_old = pd.read_excel('index_60min.xlsx', index_col=0)
# data_old.head()
# data = pd.concat([data_old, data], join='outer', sort=True)
# pd.concat([data_old, data], join='inner', sort=True).shape
# writer = pd.ExcelWriter('index_60min.xlsx')
# data.to_excel(writer)
# writer.save()

In [4]:
data_min = data[(data.index.hour == 10) | (data.index.hour == 11)].loc[:, 
    ['open', 'high', 'low', 'close', 'vol', 'amount']]

In [5]:
data_min_resample_day = data_min.resample('D').apply({
    'open':'first', 'high':'max', 'low':'min', 'close':'last',
    'vol':'sum','amount':'sum'
}).dropna()

In [6]:
data_min_resample_day.shape

(495, 6)

In [6]:
start = '2016-11-16'
end = '2018-11-23'
index_day = QA.QA_fetch_index_day_adv('000001', start, end)

In [7]:
index_day = index_day.data.xs('000001', level=1)

计算半天的涨幅和全天的涨幅

In [8]:
half_day_change = (data_min_resample_day.close / index_day.close.shift(1))[1:] - 1

In [9]:
full_day_change = index_day.close.pct_change()[1:]

验证计算是否正确

In [12]:
half_day_change.tail()

datetime
2018-11-19    0.002236
2018-11-20   -0.016327
2018-11-21   -0.001281
2018-11-22   -0.005487
2018-11-23   -0.016304
Name: close, dtype: float64

In [13]:
full_day_change.tail()

date
2018-11-19    0.009108
2018-11-20   -0.021328
2018-11-21    0.002139
2018-11-22   -0.002293
2018-11-23   -0.024930
Name: close, dtype: float64

依据半日涨幅判断全天涨幅的胜率

In [10]:
(np.sign(half_day_change) * np.sign(full_day_change)).value_counts()

 1.0    373
-1.0    121
Name: close, dtype: int64

In [80]:
373 / (373 + 121)

0.7550607287449392

我们假定当半天的涨幅越远离0，预测的准确率更高的可能性越大

In [11]:
ind = half_day_change <= 0.0004
half_filter = half_day_change[ind]
full_filter = full_day_change[ind]
result = (np.sign(half_filter) * np.sign(full_filter)).value_counts()
right_nums = (np.sign(half_filter) * np.sign(full_filter)).value_counts().loc[1.0]
total_nums = (np.sign(half_filter) * np.sign(full_filter)).value_counts().sum()
prob_fit = right_nums / total_nums

In [13]:
(np.sign(half_filter) * np.sign(full_filter)).value_counts()

 1.0    261
-1.0    114
Name: close, dtype: int64

In [89]:
(np.sign(half_filter) * np.sign(full_filter)).value_counts().sum()

382

In [12]:
prob_fit

0.703971119133574

根据Kelly criterion:

$f^* = p - {\frac{q}{b}}$

In [16]:
p = prob_fit
q = 1 - p
b = .75
f = p - q/b
f

0.30926594464500606

In [14]:
19000 * f

5453.5516347180555

In [17]:
391 / 200

1.955

In [18]:
(p - q/0.67) * 8000

2097.0957486933567

In [17]:
(q - p/1.5) * 8000

-1386.2815884476536

In [23]:
(p - q/0.8) * 19279

6437.9332129963905

In [22]:
p

0.7156862745098039

In [101]:
f_star = p/1 - q/.37
f_star

-0.10499504740342447

In [111]:
0.75 - 0.25/1.05

0.5119047619047619